In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import Dataset,DataLoader,TensorDataset
from tqdm import tqdm
import torch.nn.functional as F
from torchsummary import summary as summary

from Models.CNNmodel import CNN
from Models.ResNetmodel import resnet34
from DataUtils.load_dataset import QuickDrawDataset

In [2]:
#훈련 파라미터
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
num_epochs=30
gamma=0.1
image_size=28
learning_rate=0.1
lr_decay_step=[12,20]
momentum=0.9
weight_decay=0.0005


In [3]:
#훈련데이터
train_dataset=QuickDrawDataset(dtype='train')
train_loader=DataLoader(dataset=train_dataset,batch_size=256,shuffle=True)

#테스트 데이터
test_dataset=QuickDrawDataset(dtype='test')
test_loader=DataLoader(dataset=test_dataset,batch_size=64,shuffle=True)

num_classes=train_dataset.get_num_classes()

print("Train image num:",len(train_dataset))
print("Test image num:",len(test_dataset))



Train image num: 80000
Test image num: 20000


In [4]:
#model=CNN(num_classes).to(DEVICE)
model=resnet34(num_classes)
#loss_func=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate,
                         momentum=momentum,weight_decay=weight_decay)

C:\Users\hyj_0\anaconda3\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\hyj_0\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]             576
       BatchNorm2d-2           [-1, 64, 28, 28]             128
              ReLU-3           [-1, 64, 28, 28]               0
         MaxPool2d-4           [-1, 64, 14, 14]               0
            Conv2d-5           [-1, 64, 14, 14]          36,864
       BatchNorm2d-6           [-1, 64, 14, 14]             128
              ReLU-7           [-1, 64, 14, 14]               0
            Conv2d-8           [-1, 64, 14, 14]          36,864
       BatchNorm2d-9           [-1, 64, 14, 14]             128
             ReLU-10           [-1, 64, 14, 14]               0
       BasicBlock-11           [-1, 64, 14, 14]               0
           Conv2d-12           [-1, 64, 14, 14]          36,864
      BatchNorm2d-13           [-1, 64, 14, 14]             128
             ReLU-14           [-1, 64,

In [6]:
train_loss=0.0
train_accuracy=0.0
test_loss=0.0
test_accuracy=0.0

In [7]:
def train():
    global train_loss
    global train_accuracy
    
    model.train()
    loss_avg=0.0
    correct=0
    data_loader=tqdm(train_loader,desc='Training')

    for batch_idx,(data,target) in enumerate(data_loader):
        data,target=torch.autograd.Variable(data.cpu()),torch.autograd.Variable(target.cpu())

        data=data.view(-1,1,28,28)
        data/=255.0

        output=model(data)

        optimizer.zero_grad()
        loss=F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()

        pred=output.data.max(1)[1]
        correct=correct+float(pred.eq(target.data).sum())
        loss_avg=loss_avg*0.2+float(loss)*0.8
            
    train_loss=loss_avg
    train_accuracy=correct/len(train_loader.dataset)
    print(correct,"a:",train_accuracy)

In [8]:
def test():
    global test_loss
    global test_accuracy
    model.eval()
    loss_avg = 0.0
    correct = 0

    data_loader=tqdm(test_loader,desc='Testing')

    for batch_idx,(data,target) in enumerate(data_loader):
        data,target=torch.autograd.Variable(data.cpu()),torch.autograd.Variable(target.cpu())

        data=data.view(-1,1,28,28)
        data/=255.0

        output=model(data)

        optimizer.zero_grad()
        loss=F.cross_entropy(output,target)

        pred=output.data.max(1)[1]
        correct=correct+float(pred.eq(target.data).sum())
        
        

        loss_avg=loss_avg+float(loss)

    test_loss=loss_avg/len(test_loader)
    test_accuracy=correct/len(test_loader.dataset)
    print(correct,"a:",test_accuracy)


In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):
    print("epoch "+str(epoch+1)+" is running...")
    if epoch+1 in lr_decay_step:
        learning_rate=learning_rate*gamma
        for param_group in optimizer.param_groups:
            param_group['lr']=learning_rate
            
    current_epoch=epoch+1
    print("current epoch:",current_epoch)
    print("TRAIN")
    train()
    print("TEST")
    test()
    print("test accuracy:",test_accuracy)
    
    if test_accuracy > best_accuracy:
        best_accuracy=test_accuracy
        torch.save(model.state_dict(),os.path.join("./",'model.pytorch'))
        
    print('Best Accuracy: %.4f' %best_accuracy)

epoch 1 is running...
current epoch: 1
TRAIN


Training:  95%|██████████████████████████████████████████████████████████████████▏   | 296/313 [28:02<01:35,  5.60s/it]